## Setup and Configuration for Multi class Effnet

In [13]:
import os
import cv2
import json
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
from torchvision.models import efficientnet_b2

from sklearn.metrics import classification_report, multilabel_confusion_matrix
from sklearn.model_selection import ParameterGrid

In [14]:
# Paths
root_path = os.path.join('..', 'dataset', 'malaria')
train_json_path = os.path.join(root_path, 'training.json')
test_json_path = os.path.join(root_path, 'test.json')
image_path = os.path.join(root_path, 'images')
models_dir = os.path.join('.', 'effnet_models')

os.makedirs(models_dir, exist_ok=True)

print("Root Path:", root_path)
print("Train JSON Path:", train_json_path)
print("Test JSON Path:", test_json_path)
print("Image Path:", image_path)

Root Path: ..\dataset\malaria
Train JSON Path: ..\dataset\malaria\training.json
Test JSON Path: ..\dataset\malaria\test.json
Image Path: ..\dataset\malaria\images


## Model and Dataset Definitions

In [15]:
class MalariaDataset(Dataset):
    """
        class for mapping images with bounding boxes
    """
    def __init__(self, json_path, image_root, transform=None, category_map=None):
        with open(json_path, 'r') as f:
            self.entries = json.load(f)
        self.image_root = image_root
        self.transform = transform

        if category_map is None:
            all_categories = set()
            for item in self.entries:
                for obj in item['objects']:
                    all_categories.add(obj['category'])
            self.category_map = {cat: idx for idx, cat in enumerate(sorted(list(all_categories)))}
        else:
            self.category_map = category_map
        
        self.labels = []
        for item in self.entries:
            if item['objects']:
                cat = item['objects'][0]['category']
                self.labels.append(self.category_map[cat])
            else:
                self.labels.append(-1)

    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        entry = self.entries[idx]
        pathname_from_json = entry['image']['pathname']
        image_name = os.path.basename(pathname_from_json)
        image_full_path = os.path.join(self.image_root, image_name)

        try:
            image = Image.open(image_full_path).convert("RGB")
        except FileNotFoundError:
            print(f"Error: Image not found at {image_full_path}")
            return None # Will be filtered by collate_fn

        boxes = []
        labels = []
        for obj in entry['objects']:
            bb = obj['bounding_box']
            boxes.append([bb['minimum']['c'], bb['minimum']['r'], bb['maximum']['c'], bb['maximum']['r']])
            labels.append(self.category_map[obj['category']])

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        if self.transform:
            image = self.transform(image)

        target = {'boxes': boxes, 'labels': labels}
        return image, target

### Model architecture

In [16]:
class EfficientNetDetector(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetDetector, self).__init__()
        # Loading pre-trained EfficientNet-B2 as the backbone
        self.backbone = efficientnet_b2(weights='IMAGENET1K_V1')
        
        # Replacing the final classifier with an identity layer
        in_features = self.backbone.classifier[1].in_features
        self.backbone.classifier = nn.Identity()
        
        # classifier for multi-label classification
        self.classifier = nn.Linear(in_features, num_classes)

    def forward(self, x):
        # Pass input through the backbone
        features = self.backbone(x)
        
        # Getting class scores from the classifier
        class_scores = self.classifier(features)
        
        # return the class scores
        return class_scores

## Training and Validation Functions

In [17]:
def train_model(model, loader, optimizer, device, epoch, num_classes):
    model.train()
    running_loss, correct, total_objects = 0.0, 0, 0
    pbar = tqdm(loader, desc=f"Training Epoch {epoch}")
    
    # the loss function BCEWithLogitsLoss is ideal for multi-label tasks as it combines Sigmoid + BCE.
    # `pos_weight` forces the model to focus on rare classes by heavily penalizing mistakes on them.
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)

    for images, targets_list in pbar:
        if not images.numel(): continue # Skip empty batches
        images = images.to(device)
        optimizer.zero_grad()
        
        # Getting the class scores
        class_scores = model(images)
        
        # Creating a placeholder for all target labels in the batch
        all_target_labels = torch.zeros_like(class_scores).to(device)

        # Populate the multi-hot encoded tensor
        for i, target in enumerate(targets_list):
            labels = target['labels']
            if len(labels) > 0:
                all_target_labels[i, labels] = 1.0

        # Calculate loss for the whole batch at once
        loss = criterion(class_scores, all_target_labels)

        if images.size(0) > 0:
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Apply sigmoid to get probabilities, then threshold to get predictions
        preds = torch.sigmoid(class_scores)
        preds[preds >= 0.5] = 1
        preds[preds < 0.5] = 0

        # Compare if the predicted multi-hot vector exactly matches the true one
        total_objects += images.size(0)
        correct += (preds == all_target_labels).all(dim=1).sum().item()

        pbar.set_postfix(loss=f"{loss.item():.4f}")

    epoch_loss = running_loss / len(loader) if len(loader) > 0 else 0
    accuracy = 100 * correct / total_objects if total_objects > 0 else 0
    return epoch_loss, accuracy

def validate_model(model, loader, device, category_map, return_preds=False):
    model.eval()
    all_labels_for_report = []
    all_preds_for_report = []
    
    # Add criterion to calculate loss
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)
    running_loss = 0.0
    
    pbar = tqdm(loader, desc="Validating")

    with torch.no_grad():
        for images, targets_list in pbar:
            if not images.numel(): continue
            images = images.to(device)
            
            class_scores = model(images)
            
            # Create the true multi-hot labels for the batch
            true_labels = torch.zeros_like(class_scores)
            for i, target in enumerate(targets_list):
                labels = target['labels']
                if len(labels) > 0:
                    true_labels[i, labels] = 1.0
            
            # --- NEW: Calculate and accumulate loss ---
            loss = criterion(class_scores, true_labels)
            running_loss += loss.item()

            # Get predictions
            preds = torch.sigmoid(class_scores)
            preds[preds >= 0.5] = 1
            preds[preds < 0.5] = 0
            all_preds_for_report.extend(preds.cpu().numpy())
            all_labels_for_report.extend(true_labels.cpu().numpy())

    if return_preds:
        return all_labels_for_report, all_preds_for_report
    else:
        # --- MODIFIED: Return loss and accuracy ---
        epoch_loss = running_loss / len(loader) if len(loader) > 0 else 0
        accuracy = 100 * np.mean(np.array(all_labels_for_report) == np.array(all_preds_for_report))
        return epoch_loss, accuracy

## Main Training Pipeline

In [18]:
# Custom collate_fn to filter out None values from the batch
def custom_collate_fn(batch):
    batch = list(filter(lambda x: x is not None, batch))
    if not batch:
        return torch.tensor([]), []
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    images = torch.stack(images, dim=0)
    return images, targets

In [20]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- Pre-calculation of Class Weights ---
# This is efficient because it doesn't change between runs.
with open(train_json_path, 'r') as f:
    train_entries = json.load(f)
temp_ds = MalariaDataset(train_json_path, image_path) # Temp dataset to get map
category_map = temp_ds.category_map
num_classes = len(category_map)
class_names = [name for name, index in sorted(category_map.items(), key=lambda item: item[1])]
print(f"Found {num_classes} classes: {category_map}")

print("Calculating class weights for the loss function...")
multi_hot_labels = np.zeros((len(train_entries), num_classes), dtype=float)
for i, entry in enumerate(train_entries):
    for obj in entry['objects']:
        cat_idx = category_map.get(obj['category'])
        if cat_idx is not None:
            multi_hot_labels[i, cat_idx] = 1.0
positive_counts = multi_hot_labels.sum(axis=0)
total_samples = len(train_entries)
pos_weight = (total_samples - positive_counts) / (positive_counts + 1e-6)
pos_weight_tensor = torch.tensor(pos_weight, dtype=torch.float).to(DEVICE)
print(f"Calculated pos_weight: {pos_weight_tensor.cpu().numpy()}")


# --- Define Hyperparameter Grid for the Search ---
param_grid = {
    # 'lr': [0.001, 0.0005],
    # 'optimizer': ['Adam', 'SGD'],
    # 'batch_size': [16, 32],
    # 'image_size': [224],
    # 'sampling': ['oversample', None]
    'lr': [0.001],
    'optimizer': ['Adam'],
    'batch_size': [16],
    'image_size': [224],
    'sampling': ['oversample']
}

# --- Setup for tracking results ---
results = []
grid = list(ParameterGrid(param_grid))

Using device: cpu
Found 7 classes: {'difficult': 0, 'gametocyte': 1, 'leukocyte': 2, 'red blood cell': 3, 'ring': 4, 'schizont': 5, 'trophozoite': 6}
Calculating class weights for the loss function...
Calculated pos_weight: [ 2.5321636  7.882353  11.851064   0.         3.9508197  6.6942673
  1.0268457]


In [21]:
# Helper function for the sampler
def create_sampler(train_ds):
    class_counts = np.bincount([label for label in train_ds.labels if label != -1], minlength=num_classes)
    class_weights = 1. / (class_counts + 1e-6)
    sample_weights = np.array([class_weights[t] if t != -1 else 0 for t in train_ds.labels])
    return WeightedRandomSampler(torch.from_numpy(sample_weights).double(), len(sample_weights))

In [ ]:
# Path to the aggregated results file
results_filepath = os.path.join(models_dir, "grid_search_results.json")

def run_experiment(params):
    """Train or load a model for this set of params; append results to a single JSON."""
    print("\n" + "="*50)
    print(f"Params: {params}")
    print("="*50)

    # Build the unique model filename for this parameter combination
    sampling_str = params.get('sampling') if params.get('sampling') is not None else 'none'
    model_file = f"model_lr-{params['lr']}_optim-{params['optimizer']}_bs-{params['batch_size']}_sampling-{sampling_str}.pth"
    model_path = os.path.join(models_dir, model_file)

    # Read existing aggregated results, if any
    if os.path.isfile(results_filepath):
        with open(results_filepath, "r") as rf:
            aggregated_results = json.load(rf)
    else:
        aggregated_results = []

    # See if this parameter set has already been processed and the model exists
    existing_entry = next((e for e in aggregated_results if e.get("params") == params), None)
    if existing_entry and os.path.isfile(model_path):
        print(f"Found existing model and results for params {params}. Skipping training.")
        # Include a flag to indicate the run was skipped
        return existing_entry | {"skipped": True, "training_time_minutes": 0.0}

    start_time = time.time()

    # ----- Data preparation -----
    transform = transforms.Compose([
        transforms.Resize((params['image_size'], params['image_size'])),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    train_ds = MalariaDataset(train_json_path, image_path, transform=transform, category_map=category_map)
    val_ds   = MalariaDataset(test_json_path,  image_path, transform=transform, category_map=category_map)

    sampler, shuffle = None, True
    if params.get('sampling') == 'oversample':
        sampler = create_sampler(train_ds)
        shuffle = False

    train_loader = DataLoader(train_ds, batch_size=params['batch_size'], shuffle=shuffle, sampler=sampler, collate_fn=custom_collate_fn)
    val_loader   = DataLoader(val_ds,   batch_size=params['batch_size'], shuffle=False,              collate_fn=custom_collate_fn)

    # ----- Model and optimizer -----
    model = EfficientNetDetector(num_classes=num_classes).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=params['lr']) if params['optimizer'] == 'Adam' \
               else torch.optim.SGD(model.parameters(), lr=params['lr'], momentum=0.9)

    # ----- Training loop -----
    history = {'train_loss': [], 'train_accuracy': [], 'val_loss': [], 'val_accuracy': []}
    best_val_accuracy = 0.0
    NUM_EPOCHS = 1

    for epoch in range(1, NUM_EPOCHS + 1):
        train_loss, train_acc = train_model(model, train_loader, optimizer, DEVICE, epoch, num_classes)
        val_loss,   val_acc   = validate_model(model, val_loader, DEVICE, category_map)
        print(f"Epoch {epoch}: Train Loss {train_loss:.4f}, Train Acc {train_acc:.2f}%, Val Loss {val_loss:.4f}, Val Acc {val_acc:.2f}%")

        history['train_loss'].append(train_loss)
        history['train_accuracy'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_accuracy'].append(val_acc)

        # Save a new best model
        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            print(f"New best model! Val Accuracy: {best_val_accuracy:.2f}%")
            torch.save({
                'model_state_dict': model.state_dict(),
                'params': params,
                'best_val_accuracy': best_val_accuracy
            }, model_path)
            print(f"Model saved to {model_path}")

    # ----- Update aggregated results -----
    elapsed_time = time.time() - start_time
    # Remove any prior entry for these params
    aggregated_results = [e for e in aggregated_results if e.get("params") != params]
    aggregated_results.append({
        "params": params,
        "best_accuracy": best_val_accuracy,
        "training_time_minutes": elapsed_time / 60,
        "history": history
    })
    with open(results_filepath, "w") as rf:
        json.dump(aggregated_results, rf, indent=4)

    return {
        "params": params,
        "best_accuracy": best_val_accuracy,
        "training_time_minutes": elapsed_time / 60,
        "history": history,
        "skipped": False
    }


# Define your hyperparameter grid
param_grid = {
    'lr': [0.001, 0.0005],
    'optimizer': ['Adam', 'SGD'],
    'batch_size': [16, 32],
    'image_size': [224],
    'sampling': ['oversample', None],
}

# Run the grid search
all_results = []
for params in ParameterGrid(param_grid):
    result = run_experiment(params)
    all_results.append(result)



Params: {'batch_size': 16, 'image_size': 224, 'lr': 0.001, 'optimizer': 'Adam', 'sampling': 'oversample'}


Validating: 100%|██████████| 8/8 [00:13<00:00,  1.67s/it]


Epoch 1: Train Loss 0.3290, Train Acc 18.29%, Val Loss 0.9776, Val Acc 45.36%
New best model! Val Accuracy: 45.36%
Model saved to .\effnet_models\model_lr-0.001_optim-Adam_bs-16_sampling-oversample.pth

Params: {'batch_size': 16, 'image_size': 224, 'lr': 0.001, 'optimizer': 'Adam', 'sampling': None}


Validating: 100%|██████████| 8/8 [00:12<00:00,  1.50s/it]


Epoch 1: Train Loss 0.8273, Train Acc 6.79%, Val Loss 0.7904, Val Acc 74.52%
New best model! Val Accuracy: 74.52%
Model saved to .\effnet_models\model_lr-0.001_optim-Adam_bs-16_sampling-none.pth

Params: {'batch_size': 16, 'image_size': 224, 'lr': 0.001, 'optimizer': 'SGD', 'sampling': 'oversample'}


Training Epoch 1:  70%|██████▉   | 53/76 [04:15<01:44,  4.53s/it, loss=0.8566]